# User Management
This notebook will demonstrate the platform APIs for local user management. Note that RBAC uses a different workflow.

Additionally note that YBA can also be configured to authenticate users using LDAP [https://docs.yugabyte.com/preview/yugabyte-platform/administer-yugabyte-platform/ldap-authentication/] or OIDC [https://docs.yugabyte.com/preview/yugabyte-platform/administer-yugabyte-platform/oidc-authentication/]. The user management APIs in this notebook are not applicable in those cases. 

## Setup

Set the `YB_PLATFORM_URL` and `YB_API_KEY` environment variables in a .env file before running this notebook.

Note: "verify=False" is used to disable SSL verification for the purposes of this demonstration, but you should use appropriate certificates in a production environment.


In [ ]:
import os
import requests
from pprint import pprint
from dotenv import load_dotenv

load_dotenv()

yba_url = os.getenv("YBA_PLATFORM_URL")
yba_api_key = os.getenv("YBA_API_KEY")

headers = {
  'Content-Type': "application/json",
  'X-AUTH-YW-API-TOKEN': f"{yba_api_key}"
}

session = requests.Session()
session.verify = False
session.headers = headers

pprint(yba_url)
pprint(yba_api_key)
pprint(headers)

## Get the customer ID

The customer ID is unique to the YBA platform and is required for the following Platform API calls. 

In [ ]:
route = f"{yba_url}/api/v1/session_info"
response = session.get(url=route).json()
customer_uuid = response["customerUUID"]

print(customer_uuid)

## List current users

List users that can login to the YBA platform.

ref: https://api-docs.yugabyte.com/docs/yugabyte-platform/18e7f5bab7963-list-all-users

In [ ]:
route = f"{yba_url}/api/v1/customers/{customer_uuid}/users"
response = session.get(url=route).json()
pprint(response)

## Get details for a specific user

Choose a user from the above list and get their details. Place the user's UUID in the `user_uuid` variable below.

ref: https://api-docs.yugabyte.com/docs/yugabyte-platform/c11938226a50a-get-a-user-s-details

In [ ]:
user_uuid="asdfasdf-asdf-asdf-asdf-asdfasdfasdf" # Replace with your user UUID

route = f"{yba_url}/api/v1/customers/{customer_uuid}/users/{user_uuid}"
response = session.get(url=route).json()
pprint(response)

## Create a user

Create a new user of the YBA platform.

ref: https://api-docs.yugabyte.com/docs/yugabyte-platform/b37e8d3b835f1-create-a-user

In [ ]:
email = "name@email.tld" # Replace with user's email
password = "Yo4rPassw*rd" # Replace with user's password
confirm_password = "Yo4rPassw*rd" # Replace with user's password
role = "ReadOnly" # Replace with user's role (ConnectOnly, ReadOnly, BackupAdmin, Admin)
timezone = "America/New_York" # Replace with user's timezone

route = f"{yba_url}/api/v1/customers/{customer_uuid}/users"
payload = {
  "email": email,
  "password": password,
  "confirmPassword": confirm_password,
  "role": role,
  "timezone": timezone
}

# avoids no CSRF token error by emptying the cookie jar
session.cookies = requests.cookies.RequestsCookieJar()

response = session.post(url=route, json=payload).json()
pprint(response)

## Delete a user

Delete a user from the YBA platform.

ref: https://api-docs.yugabyte.com/docs/yugabyte-platform/9fbb314f9b10f-delete-a-user

In [ ]:
user_uuid="asdfasdf-asdf-asdf-asdf-asdfasdfasdf" # Replace with your user UUID

route = f"{yba_url}/api/v1/customers/{customer_uuid}/users/{user_uuid}"
response = session.delete(url=route).json()
pprint(response)